In [0]:
"""
!pip uninstall torchvision
!pip uninstall torch
!pip install -q torch==1.0.1 torchvision==0.2.1
"""

In [0]:
import torch
print(torch.__version__)

1.0.1


In [0]:
# Automatically reload external libraries that change
%reload_ext autoreload
%autoreload 2

# If a matplotlib plot command is issued, display the results in the notebook
%matplotlib inline

In [0]:
import numpy as np
import torch
the_seed = 42
def set_seed(seed):
    # Note, deterministic can impede performance
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)

In [0]:
# Upload data
from google.colab import files
files.upload()

Saving images_w_hits.tar.gz to images_w_hits.tar.gz
Saving images_w_truth.tar.gz to images_w_truth.tar.gz


In [0]:
!tar xzf images_w_hits.tar.gz
!tar xzf images_w_truth.tar.gz

# Upload python files and import

In [0]:
# Upload supporting python modules
!rm -f *.py
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
from data import *
from transforms import *
from unet import *
from unet_learner import *
from img_util import *
from model_util import *

# Setup

In [0]:
from torchvision import transforms
from torch.autograd import Variable
import torch
import torch.optim as opt
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [0]:
set_seed(the_seed)
batch_size=96
bunch = SegmentationBunch("Images", "Hits", "Truth", batch_size=batch_size, valid_pct = 0.3,
    transform=transforms.Compose([ToTensor(True)]))

In [0]:
def get_class_weights(n_track, n_shower, n_void):
    weights = [1. / n_void, 1. / n_shower, 1. / n_track]
    return [weight / sum(weights) for weight in weights]

n_track = 1114287
n_shower = 516073
n_void = 684138344

weights = get_class_weights(n_track, n_shower, n_void)
weights

[0.0005152961649927253, 0.6831085233826877, 0.3163761804523196]

In [0]:
# Implement OneCycleLR class vailable in later versions of torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
import math
class OneCycleLR(_LRScheduler):
    def __init__(self,
                 optimizer,
                 max_lr,
                 total_steps=None,
                 epochs=None,
                 steps_per_epoch=None,
                 pct_start=0.3,
                 anneal_strategy='cos',
                 cycle_momentum=True,
                 base_momentum=0.85,
                 max_momentum=0.95,
                 div_factor=25.,
                 final_div_factor=1e4,
                 last_epoch=-1):

        # Validate optimizer
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        # Validate total_steps
        if total_steps is None and epochs is None and steps_per_epoch is None:
            raise ValueError("You must define either total_steps OR (epochs AND steps_per_epoch)")
        elif total_steps is not None:
            if total_steps <= 0 or not isinstance(total_steps, int):
                raise ValueError("Expected non-negative integer total_steps, but got {}".format(total_steps))
            self.total_steps = total_steps
        else:
            if epochs <= 0 or not isinstance(epochs, int):
                raise ValueError("Expected non-negative integer epochs, but got {}".format(epochs))
            if steps_per_epoch <= 0 or not isinstance(steps_per_epoch, int):
                raise ValueError("Expected non-negative integer steps_per_epoch, but got {}".format(steps_per_epoch))
            self.total_steps = epochs * steps_per_epoch
        self.step_size_up = float(pct_start * self.total_steps) - 1
        self.step_size_down = float(self.total_steps - self.step_size_up) - 1

        # Validate pct_start
        if pct_start < 0 or pct_start > 1 or not isinstance(pct_start, float):
            raise ValueError("Expected float between 0 and 1 pct_start, but got {}".format(pct_start))

        # Validate anneal_strategy
        if anneal_strategy not in ['cos', 'linear']:
            raise ValueError("anneal_strategy must by one of 'cos' or 'linear', instead got {}".format(anneal_strategy))
        elif anneal_strategy == 'cos':
            self.anneal_func = self._annealing_cos
        elif anneal_strategy == 'linear':
            self.anneal_func = self._annealing_linear

        # Initialize learning rate variables
        max_lrs = self._format_param('max_lr', self.optimizer, max_lr)
        if last_epoch == -1:
            for idx, group in enumerate(self.optimizer.param_groups):
                group['initial_lr'] = max_lrs[idx] / div_factor
                group['max_lr'] = max_lrs[idx]
                group['min_lr'] = group['initial_lr'] / final_div_factor

        # Initialize momentum variables
        self.cycle_momentum = cycle_momentum
        if self.cycle_momentum:
            if 'momentum' not in self.optimizer.defaults and 'betas' not in self.optimizer.defaults:
                raise ValueError('optimizer must support momentum with `cycle_momentum` option enabled')
            self.use_beta1 = 'betas' in self.optimizer.defaults
            max_momentums = self._format_param('max_momentum', optimizer, max_momentum)
            base_momentums = self._format_param('base_momentum', optimizer, base_momentum)
            if last_epoch == -1:
                for m_momentum, b_momentum, group in zip(max_momentums, base_momentums, optimizer.param_groups):
                    if self.use_beta1:
                        _, beta2 = group['betas']
                        group['betas'] = (m_momentum, beta2)
                    else:
                        group['momentum'] = m_momentum
                    group['max_momentum'] = m_momentum
                    group['base_momentum'] = b_momentum

        super(OneCycleLR, self).__init__(optimizer, last_epoch)

    def _format_param(self, name, optimizer, param):
        """Return correctly formatted lr/momentum for each param group."""
        if isinstance(param, (list, tuple)):
            if len(param) != len(optimizer.param_groups):
                raise ValueError("expected {} values for {}, got {}".format(
                    len(optimizer.param_groups), name, len(param)))
            return param
        else:
            return [param] * len(optimizer.param_groups)

    def _annealing_cos(self, start, end, pct):
        "Cosine anneal from `start` to `end` as pct goes from 0.0 to 1.0."
        cos_out = math.cos(math.pi * pct) + 1
        return end + (start - end) / 2.0 * cos_out

    def _annealing_linear(self, start, end, pct):
        "Linearly anneal from `start` to `end` as pct goes from 0.0 to 1.0."
        return (end - start) * pct + start

    def get_lr(self):
        lrs = []
        step_num = self.last_epoch

        if step_num > self.total_steps:
            raise ValueError("Tried to step {} times. The specified number of total steps is {}"
                             .format(step_num + 1, self.total_steps))

        for group in self.optimizer.param_groups:
            if step_num <= self.step_size_up:
                computed_lr = self.anneal_func(group['initial_lr'], group['max_lr'], step_num / self.step_size_up)
                if self.cycle_momentum:
                    computed_momentum = self.anneal_func(group['max_momentum'], group['base_momentum'],
                                                         step_num / self.step_size_up)
            else:
                down_step_num = step_num - self.step_size_up
                computed_lr = self.anneal_func(group['max_lr'], group['min_lr'], down_step_num / self.step_size_down)
                if self.cycle_momentum:
                    computed_momentum = self.anneal_func(group['base_momentum'], group['max_momentum'],
                                                         down_step_num / self.step_size_down)

            lrs.append(computed_lr)
            if self.cycle_momentum:
                if self.use_beta1:
                    _, beta2 = group['betas']
                    group['betas'] = (computed_momentum, beta2)
                else:
                    group['momentum'] = computed_momentum

        return lrs

In [27]:
# Create the network, loss function and optimizer
#from torch.optim.lr_scheduler import OneCycleLR, LambdaLR
set_seed(the_seed)
num_epochs = 10
pct_start = 2. / num_epochs
n_classes = 3
wd = 1e-4
max_lr = 1e-2
base_lr = 1e-4
min_lr = 1e-7
div_factor = max_lr / base_lr
final_factor = base_lr / min_lr
model = UNet(1, n_classes = n_classes, depth = 4, n_filters = 16, y_range = (0, n_classes))
model.load_state_dict(torch.load("unet_0.pkl", map_location="cpu"))
model.eval()
loss_fn = nn.CrossEntropyLoss(torch.tensor(weights))
optim = opt.Adam(model.parameters(), lr=max_lr, weight_decay=wd)
scheduler = OneCycleLR(optim, max_lr, anneal_strategy='cos', total_steps = num_epochs * len(bunch.train_dl), pct_start=pct_start, div_factor=div_factor, final_div_factor=final_factor)
learner = UNetLearner(model, bunch, loss_fn, optim, scheduler)

Reinitialising Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Reinitialising ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
Reinitialising ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2

# Create traced script module

In [0]:
set_seed(the_seed)
datum = None
for batch in learner.bunch.valid_dl:
    image, truth = batch
    x = Variable(image)
    y = Variable(truth)
    break
sm = torch.jit.trace(learner.model, x)
sm.save("unet_0.pt")

In [0]:
files.download("unet_0.pt")